# **Atmospheric correction of Sentinel 2 image using Py6S in Google Colab environment**

This is the first part of python codes used in the article. The codes are tested inside Google Colab environment using Hong Kong water as the study area.

Guidance and reference provided at the following websites are appreciated.

*   https://github.com/samsammurphy/gee-atmcorr-S2
*   https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md
*   https://blog.csdn.net/qq_45110581/article/details/108629636








# Step 1 - Set up Py6S in Google Colab

In [23]:
#!/bin/bash

# Function to check if the directory contains only a Makefile
contains_only_makefile() {
    local dir=$1
    local file_count=$(ls -1q "$dir" | wc -l)
    if [[ $file_count -eq 1 && -f "$dir/Makefile" ]]; then
        return 0
    else
        return 1
    fi
}

# Directory and tar file variables
DIR="6SV1.1"
TAR_URL="http://rtwilson.com/downloads/6SV-1.1.tar"
TAR_FILE="6SV-1.1.tar"

# Check if the directory exists and contains only a Makefile
if [[ ! -d $DIR || ! contains_only_makefile $DIR ]]; then
    # Download and extract the tar file
    wget $TAR_URL -O $TAR_FILE
    mkdir -p $DIR
    tar xvf $TAR_FILE -C $DIR
fi

# Change to the directory and print gfortran version
cd $DIR
gfortran -v

30226.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using built-in specs.
COLLECT_GCC=gfortran
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/11/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:amdgcn-amdhsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 11.4.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-11/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,m2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-11 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --enable-libphobos-checking=release --with-target-system-zlib=auto --enable-objc-gc=auto --enable-multiarch --disab

**Manual work required before executing the subsequent code**

Refer to comments below

In [14]:
# modify "makefile" from "FC = g77 $(FFLAGS)" to "FC = gfortran -std=legacy -ffixed-line-length-none -ffpe-summary=none $(FFLAGS)"
# upload modified "makefile" to /content/6SV1.1

import os
os.getcwd()
# os.chdir(os.getcwd()+"/6SV1.1")
!echo Soodrohin28! | sudo -S make
os.environ["PATH"]=os.getcwd()+":"+os.environ["PATH"]
# test
!sixsV1.1 < ../Examples/Example_In_1.txt
!pip install Py6S
from Py6S import *
SixS.test()

[sudo] password for rohinsood: gfortran  main.o  AATSR.o ABSTRA.o AEROSO.o AKTOOL.o ATMREF.o AVHRR.o BBM.o BDM.o BRDFGRID.o CHAND.o CLEARW.o CSALBR.o DICA1.o DICA2.o DICA3.o  DISCOM.o DISCRE.o DUST.o ENVIRO.o EQUIVWL.o GAUSS.o GLI.o GOES.o HAPKALBE.o HAPKBRDF.o HRV.o IAPIALBE.o IAPIBRDF.o IAPITOOLS.o INTERP.o ISO.o KERNEL.o KERNELPOL.o LAKEW.o MAS.o MERIS.o METEO.o METH1.o METH2.o METH3.o METH4.o METH5.o METH6.o MIDSUM.o MIDWIN.o MIE.o MINNALBE.o MINNBRDF.o MOCA1.o MOCA2.o MOCA3.o MOCA4.o MOCA5.o MOCA6.o MODIS.o MSS.o NIOX1.o MODISBRDF.o MODISALBE.o NIOX2.o NIOX3.o NIOX4.o NIOX5.o NIOX6.o OCEA.o OCEAALBE.o OCEABRDF.o OCEABRDFFAST.o OCEATOOLS.o ODA550.o ODRAYL.o OS.o OSPOL.o OXYG3.o OXYG4.o OXYG5.o OXYG6.o OZON1.o PLANPOL.o POLDER.o POSGE.o POSGW.o POSLAN.o POSMTO.o POSNOA.o POSSOL.o POSSPO.o POLGLIT.o POLNAD.o PRESPLANE.o PRESSURE.o PRINT_ERROR.o RAHMALBE.o RAHMBRDF.o ROUJALBE.o ROUJBRDF.o SAND.o SCATRA.o SEAWIFS.o SOLIRR.o SOOT.o SPECINTERP.o SPLIE2.o SPLIN2.o SPLINE.o SPLINT.o STM.o 

0

# Step 2 - Define functions required in atmospheric correction

**Functions created by Sam Murphy**

Modified from https://github.com/samsammurphy/gee-atmcorr-S2

In [1]:
"""
atmospheric.py, Sam Murphy (2016-10-26)

Atmospheric water vapour, ozone and AOT from GEE

Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)

"""


import ee

class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

    # start of NEXT month
    m2 = m1.advance(1,'month')

    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()

    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.

    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)
    """

    # Point geometry required
    centroid = geom.centroid()

    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)

    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

    # water image
    water_img = ee.Image(water_ic.first())

    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)

    return water_Py6S_units



  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset

    OR

    uses our fill value (which is mean value for that latlon and day-of-year)

    """

    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):

      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

      # ozone image
      ozone_img = ee.Image(ozone_ic.first())

      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))

      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)

      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

      """

      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))

      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)

    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')

    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )

    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.

    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """

    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])


    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )

      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))

      return img


    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)

    return AOT

Import required libraries

In [7]:
import ee
from Py6S import *
from datetime import datetime
import math
import os
import sys

**Initialize Google Earth Engine session**

Need enter verification using GEE account

In [2]:
ee.Authenticate(force=True)
ee.Initialize()

grep: /proc/sys/fs/binfmt_misc/WSLInterop: No such file or directory
grep: /proc/sys/fs/binfmt_misc/WSLInterop: No such file or directory
Parameter format not correct - ""


WSL Interopability is disabled. Please enable it before using WSL.
[error] WSL Interoperability is disabled. Please enable it before using WSL.(B

Successfully saved authorization token.


**Py6S function**

In [3]:
# Define Py6S function
# Modified from https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md

def func1(img):
  S2 = ee.Image(img)
  date = S2.date()
  # top of atmosphere reflectance
  toa = S2.divide(10000)

  info = S2.getInfo()['properties']
  scene_date = datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
  solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  aot = Atmospheric.aerosol(geom,date).getInfo()

  SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
  alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
  km = alt/1000 # i.e. Py6S uses units of kilometers

  # Instantiate
  s = SixS()

  # Atmospheric constituents
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Maritime # https://github.com/robintw/Py6S/blob/master/Py6S/Params/aeroprofile.py
  s.aot550 = aot

  # Earth-Sun-satellite geometry
  s.geometry = Geometry.User()
  s.geometry.view_z = 0               # always NADIR
  s.geometry.solar_z = solar_z        # solar zenith angle
  s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
  s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
  s.altitudes.set_sensor_satellite_level()
  s.altitudes.set_target_custom_altitude(km)

  def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }
    return Wavelength(bandSelect[bandname])

  def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    return rad

  def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref

  # all wavebands
  output = S2.select('QA60')
  for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
    print(band)
    output = output.addBands(surface_reflectance(band))

  return output


# Step 3 - Remove clouds using cloud mask

In [4]:
# Remove cloud and cloud shadow
# Modified from https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

AOI_point_right = ee.Geometry.Point(114.05, 22.40)  # Define AOI location
AOI_point_left = ee.Geometry.Point(113.80, 22.40)  # Mosaic with another tile is needed to cover the study area
START_DATE = '2015-01-01'   # Define start date
END_DATE = '2021-12-31'   # Define end date
CLD_PRB_THRESH = 60   #	Cloud probability (%); values greater than are considered cloud
NIR_DRK_THRESH = 0.15   # Near-infrared reflectance; values less than are considered potential cloud shadow
CLD_PRJ_DIST = 1   # Maximum distance (km) to search for cloud shadows from cloud edges
BUFFER = 100   # Distance (m) to dilate the edge of cloud-identified objects

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def get_s2_sr_cld_col_left(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

s2_right = get_s2_sr_cld_col(AOI_point_right, START_DATE, END_DATE)
s2_left = get_s2_sr_cld_col_left(AOI_point_left, START_DATE, END_DATE)

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    # return img.select('B.*').updateMask(not_cld_shdw)
    return img.updateMask(not_cld_shdw)

s2_cloudless_right = (s2_right.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask))
s2_cloudless_left = (s2_left.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask))


/home/rohinsood/.local/lib/python3.10/site-packages/ee/deprecation.py:204: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


# Step 4 - Execute the image processing

Results are saved as GEE assets

In [21]:
# Define function to chain the above process

def preprocess(image):
  s2_boa = func1(image)
  d1 = s2_boa.clip(aoi)

  # export to asset
  fname = ee.String(d1.get('system:index')).getInfo()
  export = ee.batch.Export.image.toAsset(
      image = d1,
      description= 'S2_BOA_' + fname,
      assetId = 'projects/ee-rsrohinsood/assets/S2_Py6S_mask/' +'S2_BOA_' + fname, # Manually create image collection in GEE asset first
      region = aoi,
      scale = 10)
  export.start()
  print('exporting ' +fname + '--->done')

  # find adjacent S2 tile
  d1_date = d1.date().format('yyyy-MM-dd')
  s2_left = s2_cloudless_left.filterDate(d1.date().advance(-1,'day').format('yyyy-MM-dd'), d1.date().advance(1,'day').format('yyyy-MM-dd')).first()
  s2_left_boa = func1(s2_left)
  d2 = s2_left_boa.clip(aoi)

  # export to asset
  fname = ee.String(d2.get('system:index')).getInfo()
  export = ee.batch.Export.image.toAsset(
      image = d2,
      description= 'S2_BOA_' + fname,
      assetId = 'projects/ee-rsrohinsood/assets/S2_Py6S_mask/' +'S2_BOA_' + fname,
      region = aoi,
      scale = 10)
  export.start()
  print('exporting ' +fname + '--->done')

In [24]:
thing = ee.ImageCollection("projects/ee-rsrohinsood/assets/S2_Py6S_mask/").size()





ee.Number({
  "functionInvocationValue": {
    "functionName": "Collection.size",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "projects/ee-rsrohinsood/assets/S2_Py6S_mask/"
            }
          }
        }
      }
    }
  }
})


In [25]:
# os.chdir(os.getcwd()+"/6SV1.1")

print(os.getcwd())


# Run the preprocessing & export to asset

aoi = ee.Geometry.Polygon([[[113.800, 22.570],[113.800, 22.120],[114.514, 22.120],[114.514, 22.570]]]) # Define output AOI
geom = aoi
dates = []

s2_col = s2_cloudless_right

col_length = s2_col.size().getInfo()
print(col_length)

for i in range(0,col_length):
    print(i)
    s2_list = s2_col.toList(col_length)
    img = ee.Image(s2_list.get(i))
    d1_date_info = img.date().format('yyyy-MM-dd').getInfo()
    if d1_date_info in dates:
      continue
    dates.append(d1_date_info)
    preprocess(img)

print(dates)


/mnt/c/Users/soodr/Documents/workspaces/AI/Marine-Water-Quality-Time-Series-HK/6SV1.1
129
0
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20151023T031142_20151023T031137_T49QHE--->done
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20151023T031142_20151023T031137_T49QGE--->done
1
2
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20151122T030012_20151122T031003_T49QHE--->done
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20151122T030012_20151122T031003_T49QGE--->done
3
4
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20160101T031132_20160101T031132_T49QHE--->done
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20160101T031132_20160101T031132_T49QGE--->done
5
6
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20160530T031138_20160530T081343_T49QHE--->done
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20160530T031138_20160530T081343_T49QGE--->done
7
8
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
exporting 20160619T031132_20160619T031134_T49QHE-

# Step 5 - Mosaic the Sentinel-2 tiles

The study area is divided into 2 tiles & require mosaicking step

In [62]:
# Mosaic the two tiles created above into one mosaic

aoi = ee.Geometry.Polygon([[[113.800, 22.570],[113.800, 22.120],[114.514, 22.120],[114.514, 22.570]]])

s2_boa_col = ee.ImageCollection("projects/ee-rsrohinsood/assets/S2_Py6S_mask")
print(s2_boa_col.size().getInfo())
days = ee.Dictionary(s2_boa_col.aggregate_histogram('system:time_start')).keys().getInfo()
days = [datetime.fromtimestamp(float(s)/1000.0).strftime('%Y-%m-%d') for s in days]
days = list(dict.fromkeys(days))
print(len(days))

for i in range(0,len(days)):
  print(i)
  print(days[i])
  d = ee.Date(days[i])
  t = s2_boa_col.filterDate(d,d.advance(1,'day'))
  f = ee.Image(t.first())
  t = t.mosaic().select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])
  t = t.set('system:time_start',d.millis())
  t = t.copyProperties(f, f.propertyNames())
  t = ee.Image(t)
  fname = str(days[i])
  export = ee.batch.Export.image.toAsset(
    image = t,
    description = fname,
    assetId = 'projects/ee-rsrohinsood/assets/S2_Py6S_mask_m/' + fname,
    region = aoi,
    scale = 10)
  export.start()
  print('exporting ' +fname + '--->done')

118
59
0
2015-10-22
exporting 2015-10-22--->done
1
2015-11-21
exporting 2015-11-21--->done
2
2015-12-31
exporting 2015-12-31--->done
3
2016-05-29
exporting 2016-05-29--->done
4
2016-06-18
exporting 2016-06-18--->done
5
2016-07-28
exporting 2016-07-28--->done
6
2016-09-16
exporting 2016-09-16--->done
7
2016-09-26
exporting 2016-09-26--->done
8
2016-10-26
exporting 2016-10-26--->done
9
2016-12-15
exporting 2016-12-15--->done
10
2016-12-25
exporting 2016-12-25--->done
11
2017-01-24
exporting 2017-01-24--->done
12
2017-02-03
exporting 2017-02-03--->done
13
2017-02-13
exporting 2017-02-13--->done
14
2017-07-08
exporting 2017-07-08--->done
15
2017-07-28
exporting 2017-07-28--->done
16
2017-08-12
exporting 2017-08-12--->done
17
2017-08-17
exporting 2017-08-17--->done
18
2017-09-11
exporting 2017-09-11--->done
19
2017-09-16
exporting 2017-09-16--->done
20
2017-09-26
exporting 2017-09-26--->done
21
2017-10-01
exporting 2017-10-01--->done
22
2017-10-11
exporting 2017-10-11--->done
23
2017-10-21
